In [1]:
(in-package "SMT")
(include-book "inverter")
; cert_param: (uses-smtlink)
(value-triple (tshell-ensure))
(add-default-hints '((SMT::SMT-computed-hint clause)))

 "SMT"

TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "inverter" ...):  The ttag note
just printed to the terminal indicates a modification to ACL2.  To
avoid this warning, supply an explicit :TTAGS argument when including
the book 
"/workspaces/acl2-swf-experiments/experiments/smtlink/examples/inverter".


TTAG NOTE (for included book): Adding ttag :QUICKLISP.SHELLPOOL from book /home/acl2/books/quicklisp/shellpool.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "inverter" ...):  The ttag note
just printed to the terminal indicates a modification to ACL2.  To
avoid this warning, supply an explicit :TTAGS argument when including
the book 
"/workspaces/acl2-swf-experiments/experiments/smtlink/examples/inverter".


TTAG NOTE (for included book): Adding ttag :TSHELL from book /home/acl2/books/projects/smtlink/trusted/run.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "inverter" ...):  The ttag note
just printed 

In [2]:
(defprod ringosc3
  ((n1 sig-path-p)
   (n2 sig-path-p)
   (n3 sig-path-p)

   (inv1 inverter-p)
   (inv2 inverter-p)
   (inv3 inverter-p)))

Form:  ( TABLE ACL2::XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.

In [3]:
(define ringosc3-constraints ((r ringosc3-p))
  :returns (ok booleanp)
  (and (equal (inverter->input (ringosc3->inv1 r))
              (ringosc3->n3 r))
       (equal (inverter->output (ringosc3->inv1 r))
              (ringosc3->n1 r))
       (equal (inverter->input (ringosc3->inv2 r))
              (ringosc3->n1 r))
       (equal (inverter->output (ringosc3->inv2 r))
              (ringosc3->n2 r))
       (equal (inverter->input (ringosc3->inv3 r))
              (ringosc3->n2 r))
       (equal (inverter->output (ringosc3->inv3 r))
              (ringosc3->n3 r))
       ))


Since RINGOSC3-CONSTRAINTS is non-recursive, its admission is trivial.
We observe that the type of RINGOSC3-CONSTRAINTS is described by the
theorem 
(OR (EQUAL (RINGOSC3-CONSTRAINTS R) T) (EQUAL (RINGOSC3-CONSTRAINTS R) NIL)).
We used primitive type reasoning.

Computing the guard conjecture for RINGOSC3-CONSTRAINTS....

The non-trivial part of the guard conjecture for RINGOSC3-CONSTRAINTS
is

Goal
(IMPLIES (RINGOSC3-P R)
         (AND (INVERTER-P (RINGOSC3->INV1 R))
              (RINGOSC3-P R)
              (OR (NOT (EQUAL (INVERTER->INPUT (RINGOSC3->INV1 R))
                              (RINGOSC3->N3 R)))
                  (INVERTER-P (RINGOSC3->INV1 R)))
              (OR (NOT (EQUAL (INVERTER->INPUT (RINGOSC3->INV1 R))
                              (RINGOSC3->N3 R)))
                  (RINGOSC3-P R))
              (OR (NOT (EQUAL (INVERTER->INPUT (RINGOSC3->INV1 R))
                              (RINGOSC3->N3 R)))
                  (NOT (EQUAL (INVERTER->OUTPUT (RINGOSC3->INV1 R

In [4]:
(define ringosc3-valid ((r ringosc3-p)
                        (tr any-trace-p))
  :returns (ok booleanp)
  (b* (((unless (ringosc3-constraints r)) nil)
       (r (ringosc3-fix r))
       (i1 (ringosc3->inv1 r))
       (i1valid? (inverter-valid i1 tr))
       ((unless i1valid?) nil)
       (i2 (ringosc3->inv2 r))
       (i2valid? (inverter-valid i2 tr))
       ((unless i2valid?) nil)
       (i3 (ringosc3->inv3 r))
       (i3valid? (inverter-valid i3 tr))
       ((unless i3valid?) nil))
    t)
  )


Since RINGOSC3-VALID is non-recursive, its admission is trivial.  We
observe that the type of RINGOSC3-VALID is described by the theorem
(OR (EQUAL (RINGOSC3-VALID R TR) T) (EQUAL (RINGOSC3-VALID R TR) NIL)).
We used the :type-prescription rule INVERTER-VALID.

Computing the guard conjecture for RINGOSC3-VALID....

The non-trivial part of the guard conjecture for RINGOSC3-VALID is

Goal
(IMPLIES
 (AND (ANY-TRACE-P TR) (RINGOSC3-P R))
 (AND
  (RINGOSC3-P R)
  (OR (NOT (RINGOSC3-CONSTRAINTS R))
      (RINGOSC3-P R))
  (OR
   (NOT (RINGOSC3-CONSTRAINTS R))
   (LET ((R (RINGOSC3-FIX R)))
    (AND
     (RINGOSC3-P R)
     (LET ((I1 (RINGOSC3->INV1 R)))
      (AND
       (INVERTER-P I1)
       (ANY-TRACE-P TR)
       (LET ((I1VALID? (INVERTER-VALID I1 TR)))
         (AND (OR (NOT I1VALID?) (RINGOSC3-P R))
              (OR (NOT I1VALID?)
                  (LET ((I2 (RINGOSC3->INV2 R)))
                    (AND (INVERTER-P I2)
                         (ANY-TRACE-P TR)
                       

In [5]:
(define ringosc3-count ((r ringosc3-p)
                        (curr any-table-p))
  :returns (markers maybe-integer-p
                    :hints (("Goal" :in-theory (e/d (inverter-count
                                                     boolval)))))
  :guard-hints (("Goal"
                 :in-theory (e/d (inverter-count) ())))
  (b* ((r (ringosc3-fix r))
       (i1 (ringosc3->inv1 r))
       (i2 (ringosc3->inv2 r))
       (i3 (ringosc3->inv3 r))
       (i1-count (inverter-count i1 curr))
       ((if (equal i1-count (maybe-integer-fix nil)))
        (maybe-integer-fix nil))
       (i2-count (inverter-count i2 curr))
       ((if (equal i2-count (maybe-integer-fix nil)))
        (maybe-integer-fix nil))
       (i3-count (inverter-count i3 curr))
       ((if (equal i3-count (maybe-integer-fix nil)))
        (maybe-integer-fix nil)))
    (maybe-integer-some
     (+ (maybe-integer-some->val i1-count)
        (maybe-integer-some->val i2-count)
        (maybe-integer-some->val i3-count))))
  )


Since RINGOSC3-COUNT is non-recursive, its admission is trivial.  We
observe that the type of RINGOSC3-COUNT is described by the theorem
(OR (INTEGERP (RINGOSC3-COUNT R CURR)) (EQUAL (RINGOSC3-COUNT R CURR) NIL)).
We used the :type-prescription rules MAYBE-INTEGER-FIX$INLINE and 
MAYBE-INTEGER-SOME.

Computing the guard conjecture for RINGOSC3-COUNT....

The non-trivial part of the guard conjecture for RINGOSC3-COUNT, given
the :executable-counterparts of MAYBE-INTEGER-FIX$INLINE and MAYBE-INTEGER-P,
is

Goal
(IMPLIES
 (AND (ANY-TABLE-P CURR) (RINGOSC3-P R))
 (AND
  (RINGOSC3-P R)
  (LET ((R (RINGOSC3-FIX R)))
   (AND
    (RINGOSC3-P R)
    (LET ((I1 (RINGOSC3->INV1 R)))
     (AND
      (RINGOSC3-P R)
      (LET ((I2 (RINGOSC3->INV2 R)))
       (AND
        (RINGOSC3-P R)
        (LET ((I3 (RINGOSC3->INV3 R)))
         (AND
          (INVERTER-P I1)
          (ANY-TABLE-P CURR)
          (LET ((I1-COUNT (INVERTER-COUNT I1 CURR)))
           (AND
            (OR (EQUAL I1-COUNT NIL)
  

In [6]:
(define ringosc3-one-safe-state ((r ringosc3-p)
                                 (curr any-table-p))
  :returns (ok booleanp)
  (b* ((r (ringosc3-fix r))
       (count (ringosc3-count r curr))
       ((if (equal count (maybe-integer-fix nil))) nil))
    (equal (maybe-integer-some->val count) 1))
  )


Since RINGOSC3-ONE-SAFE-STATE is non-recursive, its admission is trivial.
We observe that the type of RINGOSC3-ONE-SAFE-STATE is described by
the theorem 
(OR (EQUAL (RINGOSC3-ONE-SAFE-STATE R CURR)
           T)
    (EQUAL (RINGOSC3-ONE-SAFE-STATE R CURR)
           NIL)).
We used primitive type reasoning.

Computing the guard conjecture for RINGOSC3-ONE-SAFE-STATE....

The non-trivial part of the guard conjecture for RINGOSC3-ONE-SAFE-STATE,
given the :executable-counterparts of MAYBE-INTEGER-FIX$INLINE and
MAYBE-INTEGER-P, is

Goal
(IMPLIES (AND (ANY-TABLE-P CURR) (RINGOSC3-P R))
         (AND (RINGOSC3-P R)
              (LET ((R (RINGOSC3-FIX R)))
                (AND (RINGOSC3-P R)
                     (ANY-TABLE-P CURR)
                     (LET ((COUNT (RINGOSC3-COUNT R CURR)))
                       (AND (OR (EQUAL COUNT NIL)
                                (MAYBE-INTEGER-P COUNT))
                            (OR (EQUAL COUNT NIL) COUNT))))))).

Q.E.D.

That completes the pro

In [7]:
(define ringosc3-one-safe-trace ((r ringosc3-p)
                                 (tr any-trace-p))
  :returns (ok booleanp)
  :measure (len tr)
  (b* ((r (ringosc3-fix r))
       ((unless (consp (any-trace-fix tr))) t)
       (first (car (any-trace-fix tr)))
       (rest (cdr (any-trace-fix tr)))
       ((unless (ringosc3-one-safe-state r first)) nil))
    (ringosc3-one-safe-trace r rest)))


For the admission of RINGOSC3-ONE-SAFE-TRACE we will use the relation
O< (which is known to be well-founded on the domain recognized by O-P)
and the measure (LEN TR).  The non-trivial part of the measure conjecture
is

Goal
(AND (O-P (LEN TR))
     (IMPLIES (AND (CONSP (ANY-TRACE-FIX TR))
                   (RINGOSC3-ONE-SAFE-STATE (RINGOSC3-FIX R)
                                            (CAR (ANY-TRACE-FIX TR))))
              (O< (LEN (CDR (ANY-TRACE-FIX TR)))
                  (LEN TR)))).
Goal'

([ A key checkpoint:

Goal'
(IMPLIES (AND (CONSP TR)
              (RINGOSC3-ONE-SAFE-STATE (RINGOSC3-FIX R)
                                       (CAR (ANY-TRACE-FIX TR))))
         (O< (LEN (CDR (ANY-TRACE-FIX TR)))
             (LEN TR)))

*1 (Goal') is pushed for proof by induction.

])

Perhaps we can prove *1 by induction.  One induction scheme is suggested
by this conjecture.  

We will induct according to a scheme suggested by (LEN TR).

This suggestion was produced using the 

In [8]:
(acl2::without-waterfall-parallelism ; MattK. mod for ACL2(p)
(defthm ringosc3-one-safe-lemma
  (implies (and (ringosc3-p r)
                (any-trace-p tr)
                (consp (any-trace-fix tr))
                (consp (any-trace-fix (cdr (any-trace-fix tr))))
                (ringosc3-constraints r)
                (inverter-valid-step (ringosc3->inv1 r)
                                     (car (any-trace-fix tr))
                                     (car (any-trace-fix (cdr (any-trace-fix tr)))))
                (inverter-valid (ringosc3->inv1 r)
                                (cdr (any-trace-fix tr)))
                (inverter-valid-step (ringosc3->inv2 r)
                                     (car (any-trace-fix tr))
                                     (car (any-trace-fix (cdr (any-trace-fix tr)))))
                (inverter-valid (ringosc3->inv2 r)
                                (cdr (any-trace-fix tr)))
                (inverter-valid-step (ringosc3->inv3 r)
                                     (car (any-trace-fix tr))
                                     (car (any-trace-fix (cdr (any-trace-fix tr)))))
                (inverter-valid (ringosc3->inv3 r)
                                (cdr (any-trace-fix tr)))
                (ringosc3-one-safe-state r (car (any-trace-fix tr))))
           (ringosc3-one-safe-state r (car (any-trace-fix (cdr (any-trace-fix
                                                                tr))))))
  :hints (("Goal"
           :smtlink
           (:fty (inverter ringosc3 any-trace any-table sig-path-list sig-path
                           sig maybe-integer)
                 :functions ((ringosc3-one-safe-trace :formals ((r ringosc3-p)
                                                                (tr any-trace-p))
                                                      :returns ((ok booleanp))
                                                      :level 1)
                             (ringosc3-valid :formals ((r ringosc3-p)
                                                       (tr any-trace-p))
                                             :returns ((ok booleanp))
                                             :level 1)
                             (inverter-valid :formals ((i inverter-p)
                                                       (tr any-trace-p))
                                             :returns ((ok booleanp))
                                             :level 1)
                             (sigs-in-bool-table :formals ((sigs sig-path-listp)
                                                           (st any-table-p))
                                                 :returns ((ok booleanp))
                                                 :level 2)
                             (sigs-in-bool-trace :formals ((sigs sig-path-listp)
                                                           (tr any-trace-p))
                                                 :returns ((ok booleanp))
                                                 :level 1)
                             (inverter-valid :formals ((inverter inverter-p)
                                                       (tr any-trace-p))
                                             :returns ((ok booleanp))
                                             :level 0)
                             ))
           )))
)



SMT !>>(MAKE-EVENT (ER-PROGN (SET-WATERFALL-PARALLELISM NIL)
                             (VALUE '(VALUE-TRIPLE NIL))))

ACL2 Observation in SET-WATERFALL-PARALLELISM:  Ignoring call to set-
waterfall-parallelism since the new value is the same as the current
value.


Summary
Form:  ( MAKE-EVENT (ER-PROGN ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
NIL


SMT !>>(DEFTHM RINGOSC3-ONE-SAFE-LEMMA
        (IMPLIES (AND (RINGOSC3-P R)
                      (ANY-TRACE-P TR)
                      (CONSP (ANY-TRACE-FIX TR))
                      (CONSP (ANY-TRACE-FIX (CDR (ANY-TRACE-FIX TR))))
                      (RINGOSC3-CONSTRAINTS R)
                      (INVERTER-VALID-STEP
                           (RINGOSC3->INV1 R)
                           (CAR (ANY-TRACE-FIX TR))
                           (CAR (ANY-TRACE-FIX (CDR (ANY-TRACE-FIX TR)))))
                      (INVERTER-VALID (RINGOSC3->INV1 R)
                                      (CDR (ANY-TRACE

In [9]:
(defthm ringosc3-one-safe
  (implies (and (ringosc3-p r)
                (any-trace-p tr)
                (consp tr)
                (ringosc3-valid r tr)
                (ringosc3-one-safe-state r (car tr)))
           (ringosc3-one-safe-trace r tr))
  :hints (("Goal"
           :induct (ringosc3-one-safe-trace r tr)
           :in-theory (e/d (ringosc3-one-safe-trace
                            ringosc3-valid
                            inverter-valid)
                           (ringosc3-one-safe-lemma)))
          ("Subgoal *1/1.1"
           :use ((:instance ringosc3-one-safe-lemma
                            (r r)
                            (tr tr)))
           )))


ACL2 Warning [Double-rewrite] in ( DEFTHM RINGOSC3-ONE-SAFE ...): 
In a :REWRITE rule generated from RINGOSC3-ONE-SAFE, equivalence relations
ACL2::SET-EQUIV and ACL2::LIST-EQUIV are maintained at one problematic
occurrence of variable TR in the third hypothesis, but not at any binding
occurrence of TR.  Consider replacing that occurrence of TR in the
third hypothesis with (DOUBLE-REWRITE TR).  See :doc double-rewrite
for more information on this issue.


ACL2 Warning [Double-rewrite] in ( DEFTHM RINGOSC3-ONE-SAFE ...): 
In a :REWRITE rule generated from RINGOSC3-ONE-SAFE, equivalence relation
ACL2::LIST-EQUIV is maintained at one problematic occurrence of variable
TR in the fifth hypothesis, but not at any binding occurrence of TR.
Consider replacing that occurrence of TR in the fifth hypothesis with
(DOUBLE-REWRITE TR).  See :doc double-rewrite for more information
on this issue.


*1 (the initial Goal, a key checkpoint) is pushed for proof by induction.

We have been told to use in